# OpenAI Agents SDK - 에러 핸들링과 디버깅

이 튜토리얼은 OpenAI Agents SDK를 프로덕션 환경에서 안정적으로 운영하기 위한 에러 핸들링과 디버깅 기법을 다룬다. 에이전트 실행 중 발생할 수 있는 다양한 예외 상황을 처리하고, 실행 결과를 분석하여 성능을 최적화하는 방법을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 에러 핸들링 에이전트

이 챕터에서는 프로덕션 환경에서 필수적인 에러 핸들링 패턴을 다룬다. 에이전트 실행 중 발생할 수 있는 다양한 예외 유형을 이해하고, 이를 적절히 처리하여 시스템의 안정성을 확보하는 방법을 학습한다.

### 학습 내용

- `AgentsException` 계열 예외 유형 이해 및 처리 (MaxTurnsExceeded, ModelBehaviorError)
- 지수 백오프를 활용한 재시도 로직 구현
- 서비스 장애 시 우아한 성능 저하 처리
- 포괄적인 로깅 및 오류 보고 체계
- 프로덕션 환경을 위한 안전한 에이전트 실행 패턴

### 환경 설정 및 예외 클래스 임포트

OpenAI Agents SDK는 다양한 예외 클래스를 제공한다. `AgentsException`은 모든 SDK 관련 예외의 기본 클래스이며, `MaxTurnsExceeded`는 에이전트가 지정된 최대 턴 수를 초과했을 때 발생하고, `ModelBehaviorError`는 모델의 비정상적인 동작이 감지되었을 때 발생한다. 이러한 예외들을 적절히 처리하면 안정적인 에이전트 시스템을 구축할 수 있다.

In [2]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings
from agents.exceptions import MaxTurnsExceeded, ModelBehaviorError, AgentsException

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 기본 에러 핸들링 - 정상 실행 케이스

가장 기본적인 에러 핸들링 패턴은 try-except 블록을 사용하는 것이다. 정상적인 실행의 경우 결과를 반환하고, 예외 발생 시 적절한 메시지를 출력한다. `max_turns` 파라미터를 설정하여 에이전트의 최대 실행 턴 수를 제한할 수 있으며, 이는 무한 루프 방지에 효과적이다.

In [3]:
# 엄격한 지시사항을 가진 에이전트 생성
careful_agent = Agent(
    name="CarefulAssistant",
    model=MODEL,
    model_settings=ModelSettings(temperature=0.1),
    instructions="""
    당신은 신중한 어시스턴트로서 도움이 되는 응답을 제공합니다.
    항상 사용자 질문에 완전하고 사려 깊은 답변을 제공하세요.
    """
)

# 예제 1: 정상적인 성공 실행
print("=== 예제 1: 정상 실행 ===")
try:
    result = Runner.run_sync(
        careful_agent, 
        "삼원색은 무엇인가요?",
        max_turns=3  # 턴 수 제한
    )
    print(f"성공: {result.final_output}")
    
except AgentsException as e:
    print(f"에이전트 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류: {e}")

=== 예제 1: 정상 실행 ===
예상치 못한 오류: AgentRunner.run_sync() cannot be called when an event loop is already running.


### 최대 턴 수 초과 보호

에이전트가 복잡한 대화나 반복적인 질문으로 인해 무한 루프에 빠지는 것을 방지하기 위해 `max_turns` 파라미터를 사용한다. 이 값을 초과하면 `MaxTurnsExceeded` 예외가 발생한다. 이 예외를 포착하여 적절히 처리하면 시스템 리소스를 보호하고 비용을 절감할 수 있다.

In [4]:
print("\n" + "="*50 + "\n")
print("=== 예제 2: 최대 턴 수 보호 ===")

# 루프에 빠질 수 있는 에이전트 생성
potentially_loopy_agent = Agent(
    name="PotentiallyLoopyAgent",
    instructions="""
    당신은 때때로 후속 질문을 하는 에이전트입니다.
    사용자가 복잡한 주제에 대해 질문하면, 완전한 답변을 제공하기 전에
    명확히 하기 위한 질문을 해야 할 수도 있습니다.
    """
)

try:
    result = await Runner.run(
        potentially_loopy_agent,
        "양자 컴퓨팅에 대해 혼란스럽게 설명하고 저에게 질문을 해주세요",
        max_turns=2  # 보호 기능 시연을 위한 매우 낮은 제한
    )
    print(f"허용된 턴 내 완료: {result.final_output}")
    
except MaxTurnsExceeded as e:
    print(f"[경고] 최대 턴 수 초과: {e}")
    print("   이는 에이전트 대화에서 무한 루프를 방지한다.")
    
except AgentsException as e:
    print(f"에이전트 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류: {e}")



=== 예제 2: 최대 턴 수 보호 ===
허용된 턴 내 완료: 좋아요! 양자 컴퓨팅은 고전 컴퓨터가 0과 1만을 다루는 비트 대신, 큐비트라는 것을 사용해서 정보를 처리하고 저장합니다. 큐비트는 동시에 0이면서 1일 수 있는 중첩(superposition) 상태를 가질 수 있고, 여러 큐비트가 얽힘(entanglement)이라는 현상을 보이기도 해요. 그래서 양자 컴퓨터는 고전 컴퓨터로는 정말 오래 걸리는 계산도 빠르게 해결할 수 있을 거라고 기대됩니다. 하지만, 큐비트는 디코히런스(decoherence) 때문에 외부 환경에 굉장히 민감해서 안정적으로 제어하기 어렵습니다. 게이트(quantum gate)도 다르고 얽혀 있음 때문에 관측하기도 힘들고요. 펌블 펌블... 아, 우리가 양자 컴퓨터로 뭘 할 수 있을지, 실제로 어떻게 만드는지, 혹은 큐비트가 왜 그렇게 특별한지... 저도 아직 완전히는 모르겠네요!

혹시, 양자 컴퓨팅에서 가장 궁금한 부분이 있으세요?  
(예를 들어: 큐비트가 무엇인지, 실제로 어떻게 계산하는지, 아니면 왜 속도가 빠른지 등)


### API 오류의 우아한 처리

잘못된 모델명이나 네트워크 문제 등으로 인해 API 오류가 발생할 수 있다. 이러한 시스템 수준의 오류를 적절히 처리하면 사용자에게 유용한 피드백을 제공하고, 시스템의 전체적인 안정성을 유지할 수 있다. `ModelBehaviorError`는 모델 구성 문제를, 일반 `Exception`은 네트워크나 API 키 관련 문제를 나타낼 수 있다.

In [5]:
print("\n" + "="*50 + "\n")
print("=== 예제 3: API 오류 처리 ===")

try:
    # 오류를 유발하기 위해 잘못된 모델명으로 에이전트 생성
    faulty_agent = Agent(
        name="FaultyAgent",
        model="nonexistent-model-xyz",  # 존재하지 않는 모델
        instructions="당신은 도움이 되는 어시스턴트입니다."
    )
    
    result = await Runner.run(faulty_agent, "안녕하세요")
    print(f"예상치 못한 성공: {result.final_output}")
    
except ModelBehaviorError as e:
    print(f"[경고] 모델 동작 오류: {e}")
    print("   이는 모델 구성 문제로 인해 발생할 수 있다.")
    
except AgentsException as e:
    print(f"일반 에이전트 오류: {e}")
    print("   이는 다양한 SDK 관련 오류를 포함한다.")
    
except Exception as e:
    print(f"시스템 오류: {e}")
    print("   이는 네트워크, API 키 또는 기타 시스템 문제일 수 있다.")



=== 예제 3: API 오류 처리 ===


Error getting response: Error code: 400 - {'error': {'message': "The requested model 'nonexistent-model-xyz' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}. (request_id: req_0506756938b54a59a0ac925c6b655f24)


시스템 오류: Error code: 400 - {'error': {'message': "The requested model 'nonexistent-model-xyz' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}
   이는 네트워크, API 키 또는 기타 시스템 문제일 수 있다.


### 프로덕션 환경을 위한 안전한 실행 패턴

프로덕션 환경에서는 단순한 try-except를 넘어 재시도 로직이 필요하다. 아래의 `safe_agent_execution` 함수는 지정된 횟수만큼 재시도를 수행하며, 각 예외 유형에 따라 다른 처리 전략을 적용한다. `MaxTurnsExceeded`나 일반 시스템 오류는 재시도 가능하지만, `AgentsException`과 같은 SDK 특정 오류는 재시도해도 해결되지 않을 가능성이 높으므로 즉시 실패 처리한다.

In [6]:
print("\n" + "="*50 + "\n")
print("=== 예제 4: 프로덕션 에러 핸들링 패턴 ===")

def safe_agent_execution(agent: Agent, prompt: str, max_retries: int = 3):
    """
    재시도 기능을 갖춘 프로덕션용 안전한 에이전트 실행 패턴이다.
    
    Args:
        agent: 실행할 에이전트
        prompt: 사용자 프롬프트
        max_retries: 최대 재시도 횟수
    
    Returns:
        성공 시 RunResult, 실패 시 None
    """
    for attempt in range(max_retries):
        try:
            print(f"시도 {attempt + 1}/{max_retries}...")
            
            result = Runner.run_sync(
                agent, 
                prompt,
                max_turns=5  # 합리적인 턴 제한
            )
            
            print(f"[성공] 시도 {attempt + 1}에서 완료")
            return result
            
        except MaxTurnsExceeded:
            print(f"[경고] 시도 {attempt + 1}: 최대 턴 수 초과")
            if attempt == max_retries - 1:
                print("[실패] 최대 턴 수로 인해 모든 시도 소진")
                return None
                
        except ModelBehaviorError as e:
            print(f"[경고] 시도 {attempt + 1}: 모델 동작 오류: {e}")
            if attempt == max_retries - 1:
                print("[실패] 모델 문제로 인해 모든 시도 소진")
                return None
                
        except AgentsException as e:
            print(f"[실패] 에이전트 오류 (재시도 안함): {e}")
            return None
            
        except Exception as e:
            print(f"[경고] 시도 {attempt + 1}: 시스템 오류: {e}")
            if attempt == max_retries - 1:
                print("[실패] 시스템 오류로 인해 모든 시도 소진")
                return None
    
    return None



=== 예제 4: 프로덕션 에러 핸들링 패턴 ===


In [7]:
# 프로덕션 패턴 테스트
production_agent = Agent(
    name="ProductionAgent",
    instructions="당신은 신뢰할 수 있는 프로덕션 어시스턴트입니다."
)

result = safe_agent_execution(
    production_agent,
    "프랑스의 수도는 어디인가요? 간단히 답해주세요."
)

if result:
    print(f"\n최종 결과: {result.final_output}")
else:
    print("\n모든 재시도 후에도 결과를 얻지 못했다.")

시도 1/3...
[경고] 시도 1: 시스템 오류: AgentRunner.run_sync() cannot be called when an event loop is already running.
시도 2/3...
[경고] 시도 2: 시스템 오류: AgentRunner.run_sync() cannot be called when an event loop is already running.
시도 3/3...
[경고] 시도 3: 시스템 오류: AgentRunner.run_sync() cannot be called when an event loop is already running.
[실패] 시스템 오류로 인해 모든 시도 소진

모든 재시도 후에도 결과를 얻지 못했다.


---

## 2. 실행 결과 분석

이 챕터에서는 `RunResult` 객체를 통해 에이전트 실행 결과를 상세히 분석하는 방법을 다룬다. 실행 메타데이터와 API 사용량을 모니터링하여 에이전트의 성능을 최적화할 수 있다.

### 학습 내용

- `final_output`을 통한 주요 응답 접근
- 실행 메타데이터 검사 (에이전트 정보, 턴 수)
- API 사용량 및 토큰 소비 모니터링
- 에이전트 응답 구조 이해

### RunResult 객체의 구조

`RunResult` 객체는 에이전트 실행에 관한 모든 정보를 담고 있다. `final_output`은 에이전트의 최종 응답 텍스트를 포함하고, `last_agent`는 마지막으로 실행된 에이전트의 정보를 제공한다. `raw_responses`는 각 턴에서의 원시 API 응답을 담고 있어 세부적인 분석이 가능하다. `new_items`는 실행 과정에서 생성된 새로운 항목들의 목록이다.

In [8]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

In [10]:
# 분석용 에이전트 생성
agent = Agent(
    name="Analyzer", 
    instructions="주어진 입력을 분석하세요."
)

# 에이전트 실행
result = await Runner.run(agent, "재생 에너지의 장점은 무엇인가요?")

### 주요 출력 및 메타데이터 접근

`final_output` 속성은 에이전트가 생성한 최종 응답 텍스트를 반환한다. `last_agent.name`을 통해 실행된 에이전트의 이름을 확인할 수 있으며, 이는 멀티 에이전트 시스템에서 어떤 에이전트가 최종 응답을 생성했는지 추적하는 데 유용하다. `raw_responses`의 길이는 총 API 호출 횟수(턴 수)를 나타낸다.

In [11]:
# 주요 출력
print("=== 주요 출력 ===")
print(f"최종 응답: {result.final_output}")

# 실행 메타데이터
print("\n=== 실행 메타데이터 ===")
print(f"실행 에이전트: {result.last_agent.name}")
print(f"총 턴 수: {len(result.raw_responses)}")
print(f"생성된 새 항목 수: {len(result.new_items)}")

=== 주요 출력 ===
최종 응답: 재생 에너지의 장점은 다음과 같습니다:

1. **환경 친화적**  
   탄소 배출 및 오염물질이 적어 지구 온난화와 대기 오염을 줄이는 데 도움이 됩니다.

2. **자원 고갈 문제 해결**  
   석유, 석탄처럼 한정된 자원이 아니기 때문에 반영구적으로 사용 가능합니다.

3. **에너지 자립**  
   각국이 자체적으로 생산할 수 있어 에너지 수입에 대한 의존도를 줄일 수 있습니다.

4. **경제적 이점**  
   장기적으로 유지비가 저렴하고, 신기술 및 일자리 창출 효과가 큽니다.

5. **안전성**  
   원자력 발전과 달리 방사능 누출, 대형 사고 위험이 적습니다.

6. **지역 발전**  
   소규모 발전소로 지역 경제 활성화와 전력 분산 공급이 가능합니다.

이처럼 재생 에너지는 환경, 경제, 사회 전반에 긍정적인 영향을 주는 에너지입니다.

=== 실행 메타데이터 ===
실행 에이전트: Analyzer
총 턴 수: 1
생성된 새 항목 수: 1


### API 사용량 정보 분석

각 API 호출의 토큰 사용량을 추적하는 것은 비용 관리에 필수적이다. `raw_responses` 리스트를 순회하면서 각 턴에서 사용된 토큰 수를 확인할 수 있다. `usage.total_tokens`는 해당 턴에서 사용된 총 토큰 수를 나타내며, 이를 합산하여 전체 실행의 총 토큰 소비량을 계산할 수 있다.

In [12]:
# API 사용량 정보
print("\n=== API 사용량 정보 ===")
if result.raw_responses:
    total_tokens = 0
    for i, response in enumerate(result.raw_responses):
        usage = response.usage
        total_tokens += usage.total_tokens
        print(f"턴 {i+1}: {usage.total_tokens} 토큰 (입력: {usage.input_tokens}, 출력: {usage.output_tokens})")
    print(f"\n총 토큰 사용량: {total_tokens}")


=== API 사용량 정보 ===
턴 1: 267 토큰 (입력: 30, 출력: 237)

총 토큰 사용량: 267


---

## 3. 에이전트 실행 디버깅

이 챕터에서는 에이전트 실행을 심층적으로 분석하는 디버깅 기법을 다룬다. 실행 흐름과 성능을 이해하고 문제를 해결하는 데 필수적인 기술이다.

### 학습 내용

- `RunResult` 객체를 분석하는 디버그 함수 작성
- 여러 API 호출에 걸친 토큰 사용량 모니터링
- 가드레일 활성화 및 안전 조치 추적
- 에이전트 실행 흐름 및 성능 이해

### 종합 디버그 함수 작성

개발 과정에서 에이전트의 동작을 상세히 분석하기 위해 디버그 함수를 작성하는 것이 유용하다. 이 함수는 `RunResult` 객체를 받아 출력 길이, API 호출 횟수, 각 턴의 토큰 사용량, 가드레일 활성화 여부 등을 출력한다. 가드레일은 입력과 출력에 대한 안전 검사를 수행하는 기능으로, `input_guardrail_results`와 `output_guardrail_results`를 통해 확인할 수 있다.

In [13]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, RunResult

In [14]:
def debug_agent_execution(result: RunResult):
    """
    에이전트 실행에 대한 상세 디버깅 정보를 출력한다.
    
    Args:
        result: 에이전트 실행 결과 객체
    """
    print("=== 에이전트 실행 디버그 ===")
    print(f"최종 출력 길이: {len(result.final_output or '')} 문자")
    print(f"총 API 호출 수: {len(result.raw_responses)}")
    
    # 각 턴별 상세 정보
    total_tokens = 0
    for i, response in enumerate(result.raw_responses):
        tokens = response.usage.total_tokens
        total_tokens += tokens
        print(f"\n턴 {i+1}:")
        print(f"  토큰: {tokens} (입력: {response.usage.input_tokens}, 출력: {response.usage.output_tokens})")
    
    print(f"\n총 토큰 사용량: {total_tokens}")
    
    # 가드레일 정보
    if result.input_guardrail_results:
        print(f"\n입력 가드레일 활성화: {len(result.input_guardrail_results)}개")
    else:
        print("\n입력 가드레일: 활성화되지 않음")
    
    if result.output_guardrail_results:
        print(f"출력 가드레일 활성화: {len(result.output_guardrail_results)}개")
    else:
        print("출력 가드레일: 활성화되지 않음")
    
    print("\n" + "="*40)

### 디버그 함수 활용

디버그 에이전트를 생성하고 실행한 후, 앞서 작성한 디버그 함수를 사용하여 실행 결과를 분석한다. 이를 통해 에이전트가 얼마나 많은 턴을 사용했는지, 각 턴에서 얼마나 많은 토큰을 소비했는지, 가드레일이 활성화되었는지 등을 한눈에 파악할 수 있다.

In [15]:
# 테스트용 에이전트 생성
agent = Agent(
    name="DebugAgent",
    instructions="당신은 디버깅 목적의 도움이 되는 어시스턴트입니다."
)

# 에이전트 실행
result = await Runner.run(agent, "인공지능의 미래에 대해 간략히 설명해주세요.")

# 디버깅 함수 사용
debug_agent_execution(result)

=== 에이전트 실행 디버그 ===
최종 출력 길이: 284 문자
총 API 호출 수: 1

턴 1:
  토큰: 187 (입력: 42, 출력: 145)

총 토큰 사용량: 187

입력 가드레일: 활성화되지 않음
출력 가드레일: 활성화되지 않음



### 복잡한 쿼리에 대한 디버깅

더 복잡한 쿼리를 실행하여 에이전트의 동작을 분석해 본다. 복잡한 질문일수록 더 많은 토큰을 사용하고, 경우에 따라 여러 턴이 필요할 수 있다. 디버그 함수를 통해 이러한 패턴을 파악하고 최적화 포인트를 찾을 수 있다.

In [17]:
# 더 복잡한 쿼리로 테스트
complex_result = await Runner.run(
    agent, 
    "머신러닝, 딥러닝, 강화학습의 차이점을 비교하고 각각의 실제 적용 사례를 설명해주세요."
)

print("=== 복잡한 쿼리 디버깅 ===")
debug_agent_execution(complex_result)

print("\n=== 최종 응답 ===")
print(complex_result.final_output)

=== 복잡한 쿼리 디버깅 ===
=== 에이전트 실행 디버그 ===
최종 출력 길이: 1706 문자
총 API 호출 수: 1

턴 1:
  토큰: 850 (입력: 57, 출력: 793)

총 토큰 사용량: 850

입력 가드레일: 활성화되지 않음
출력 가드레일: 활성화되지 않음


=== 최종 응답 ===
물론입니다! 머신러닝, 딥러닝, 강화학습은 인공지능 분야의 중요한 하위 분야들이지만, 각자의 원리와 적용 분야에서 차이가 있습니다.

---

# 1. 머신러닝(Machine Learning)

### 개념
- **정의:** 데이터에서 자동으로 패턴을 학습해 예측이나 분류 등의 작업을 수행하는 기술.
- **방식:** 사람이 설계한 특징(feature)을 활용하여 기존의 알고리즘(선형 회귀, 의사결정나무, SVM 등)으로 데이터를 분석, 학습.

### 실제 적용 사례
- **스팸 이메일 분류**: 이메일 내용을 분석해 스팸/정상 메일 분류
- **신용카드 이상 거래 감지**: 거래 패턴을 학습하고, 이상 값을 찾아내 사기 탐지
- **주가 예측**: 과거 데이터를 바탕으로 미래 가격 예측 (기본적인 회귀 모델 등)

---

# 2. 딥러닝(Deep Learning)

### 개념
- **정의:** 머신러닝의 한 분야로, 인공신경망(특히 여러 층을 가진 딥러닝 구조)를 이용해 높은 수준의 추상적 패턴을 학습하는 기법.
- **방식:** 특징(feature)도 스스로 추출하며, 대량의 데이터와 연산을 필요로 함. 이미지, 소리, 텍스트 등 비정형 데이터 처리에 강점.

### 실제 적용 사례
- **이미지 인식**: 얼굴 인식, 사물 인식(CNN 등 활용)
- **음성 인식**: 스마트폰의 음성 비서(Siri, Google Assistant 등)
- **자연어 처리**: 번역기, 챗봇, 감정 분석(BERT, GPT 등)

---

# 3. 강화학습(Reinforcement Learning)

### 개념
- **정의:** 에이전트가 환경과 상호작용하며, 보

### 4 성능 비교 분석

여러 쿼리의 실행 결과를 비교하여 에이전트의 성능 패턴을 분석한다. 이를 통해 어떤 유형의 쿼리가 더 많은 리소스를 소비하는지, 응답 길이와 토큰 사용량 간의 관계는 어떠한지 등을 파악할 수 있다.

In [18]:
def compare_executions(results: list, labels: list):
    """
    여러 실행 결과를 비교 분석한다.
    
    Args:
        results: RunResult 객체 리스트
        labels: 각 결과에 대한 레이블 리스트
    """
    print("=== 실행 결과 비교 ===")
    print(f"{'쿼리':<20} {'출력 길이':<12} {'턴 수':<8} {'총 토큰':<10}")
    print("-" * 50)
    
    for result, label in zip(results, labels):
        output_len = len(result.final_output or '')
        turns = len(result.raw_responses)
        total_tokens = sum(r.usage.total_tokens for r in result.raw_responses)
        print(f"{label:<20} {output_len:<12} {turns:<8} {total_tokens:<10}")

# 다양한 쿼리 실행
simple_result = await Runner.run(agent, "안녕하세요")
medium_result = await Runner.run(agent, "파이썬의 주요 특징 3가지를 설명해주세요")

# 비교 분석
compare_executions(
    [simple_result, medium_result, complex_result],
    ["간단한 인사", "중간 질문", "복잡한 비교"]
)

=== 실행 결과 비교 ===
쿼리                   출력 길이        턴 수      총 토큰      
--------------------------------------------------
간단한 인사               58           1        64        
중간 질문                447          1        281       
복잡한 비교               1706         1        850       


---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 에러 핸들링과 디버깅 기법을 다루었다. 다음과 같은 내용을 학습하였다:

1. **에러 핸들링**: `MaxTurnsExceeded`, `ModelBehaviorError`, `AgentsException` 등 다양한 예외 유형의 처리 방법
2. **재시도 로직**: 프로덕션 환경을 위한 안전한 에이전트 실행 패턴과 재시도 전략
3. **결과 분석**: `RunResult` 객체를 통한 실행 메타데이터 및 API 사용량 모니터링
4. **디버깅 기법**: 종합적인 디버그 함수 작성과 성능 비교 분석

이러한 패턴을 적용하면 안정적이고 유지보수가 용이한 에이전트 시스템을 구축할 수 있다. 특히 프로덕션 환경에서는 적절한 에러 핸들링과 모니터링이 시스템의 신뢰성을 크게 향상시킨다.